In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('p3_reddit.csv')

In [ ]:
data.head()

,Unnamed: 0,text,label
0,0,alway taught peopl like jordan peterson matt w...,0
1,1,don’t actual know qanon hundr leagu past stand...,0
2,2,f becom concern cousin’ f well relationship cl...,0
3,3,marri narcissist man littl decad final let go ...,0
4,4,f brotherinlaw alway total jackass marri siste...,0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size=0.2, random_state=42)
X_train = X_train.astype(str)
X_test = X_test.astype(str)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, Embedding, LSTM, Dense, Dropout, Conv1D, MaxPooling1D, GlobalMaxPooling1D

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
X_train_tagged_texts = []
for text in X_train:
    tokens = word_tokenize(text)
    tags = pos_tag(tokens)
    X_train_tagged_texts.append(tags)
X_test_tagged_texts = []
for text in X_test:
    tokens = word_tokenize(text)
    tags = pos_tag(tokens)
    X_test_tagged_texts.append(tags)
# Map POS tags to integers using NLTK's tagset
tagset = {"CC": 1, "CD": 2, "DT": 3, "EX": 4, "FW": 5, "IN": 6, "JJ": 7, "JJR": 8, "JJS": 9, "LS": 10, "MD": 11,
          "NN": 12, "NNS": 13, "NNP": 14, "NNPS": 15, "PDT": 16, "POS": 17, "PRP": 18, "PRP$": 19, "RB": 20,
          "RBR": 21, "RBS": 22, "RP": 23, "SYM": 24, "TO": 25, "UH": 26, "VB": 27, "VBD": 28, "VBG": 29,
          "VBN": 30, "VBP": 31, "VBZ": 32, "WDT": 33, "WP": 34, "WP$": 35, "WRB": 36}
X_train_tagged = []
for tagged_text in X_train_tagged_texts:
    tagged_text_int = []
    for word, tag in tagged_text:
        if tag in tagset:
            tagged_text_int.append(tagset[tag])
    X_train_tagged.append(tagged_text_int)

X_test_tagged = []
for tagged_text in X_test_tagged_texts:
    tagged_text_int = []
    for word, tag in tagged_text:
        if tag in tagset:
            tagged_text_int.append(tagset[tag])
    X_test_tagged.append(tagged_text_int)


max_length = max([len(seq) for seq in X_train_tagged])
max_length = max([len(seq) for seq in X_test_tagged])
# Pad sequences to a fixed length
# max_length = 12270
X_train_padded = pad_sequences(X_train_tagged, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_tagged, maxlen=max_length, padding='post')

In [ ]:

# LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tagset)+1, output_dim=300, input_length=max_length))
model.add(Conv1D(64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Conv1D(32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(Dropout(0.2))
model.add(MaxPooling1D(pool_size=5))
model.add(Dense(32, activation='tanh'))
model.add(GlobalMaxPooling1D())
model.add(Dense(11, activation='softmax'))  # 2 classes: generated by same method or not
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# early_stop = EarlyStopping(monitor='loss', patience=3, verbose=1)
# train the model
model.fit(X_train_padded, y_train, epochs=15, batch_size=32)


Epoch 1/15
38/38 [==============================] - 30s 598ms/step - loss: 1.2293 - accuracy: 0.5406
Epoch 2/15
38/38 [==============================] - 23s 599ms/step - loss: 0.7264 - accuracy: 0.6954
Epoch 3/15
38/38 [==============================] - 23s 608ms/step - loss: 0.6491 - accuracy: 0.7161
Epoch 4/15
38/38 [==============================] - 23s 604ms/step - loss: 0.6217 - accuracy: 0.7202
Epoch 5/15
38/38 [==============================] - 23s 605ms/step - loss: 0.6125 - accuracy: 0.7310
Epoch 6/15
38/38 [==============================] - 23s 602ms/step - loss: 0.5935 - accuracy: 0.7260
Epoch 7/15
38/38 [==============================] - 22s 588ms/step - loss: 0.5697 - accuracy: 0.7392
Epoch 8/15
38/38 [==============================] - 23s 598ms/step - loss: 0.5592 - accuracy: 0.7541
Epoch 9/15
38/38 [==============================] - 23s 606ms/step - loss: 0.5524 - accuracy: 0.7599
Epoch 10/15
38/38 [==============================] - 23s 603ms/step - loss: 0.5316 - accura

In [ ]:
# tokenizer.fit_on_texts(X_test)
loss, acc = model.evaluate(X_test_padded, y_test)

10/10 [==============================] - 3s 143ms/step - loss: 0.5591 - accuracy: 0.7525


In [ ]:
preds = model.predict(X_test_padded)

10/10 [==============================] - 3s 141ms/step


In [ ]:
preds[0]

array([6.0463804e-03, 8.8920170e-01, 1.0159624e-01, 1.5347873e-04,
       3.6255713e-04, 1.3751168e-03, 2.7021364e-04, 5.9401838e-05,
       2.6019497e-04, 1.4402889e-04, 5.3065550e-04], dtype=float32)

In [ ]:
preds_new = np.argmax(preds, axis = 1)

In [ ]:
preds_new = np.array(preds_new)

In [ ]:
preds_new[0]

1

In [ ]:
from sklearn.metrics import f1_score,precision_score, recall_score, accuracy_score
f1 = f1_score(y_test, preds_new, average='macro')
print("F1 score = ",f1)
precision = precision_score(y_test, preds_new, average='macro')
recall = recall_score(y_test, preds_new, average='macro')
print("Precision = ", precision)
print("Recall = ", recall)
print("Accuracy = ", accuracy_score(y_test, preds_new))

F1 score =  0.7410393528040586
Precision =  0.7557716461006591
Recall =  0.7494030956493708
Accuracy =  0.7524752475247525


In [ ]:
np.unique(data['label'])

array([0, 1, 2])

In [ ]:
from sklearn.metrics import classification_report, roc_curve, auc, f1_score, accuracy_score, confusion_matrix
matrix = confusion_matrix(y_test, preds_new, labels = [0, 1, 2])
mat = matrix.diagonal()/matrix.sum(axis=1)
print(classification_report(y_test, preds_new, labels = [0, 1, 2],digits=4))
print('confusion matrix: ', mat)

              precision    recall  f1-score   support

           0     0.8559    0.9806    0.9140       103
           1     0.7727    0.5152    0.6182        99
           2     0.6387    0.7525    0.6909       101

    accuracy                         0.7525       303
   macro avg     0.7558    0.7494    0.7410       303
weighted avg     0.7563    0.7525    0.7430       303

confusion matrix:  [0.98058252 0.51515152 0.75247525]
